# Predict on new data using a trained CNN on XPS data on Google Colab

In this notebook, we will use a trained convolutional network to predict on unseen XPS spectra.

## Setup

### Set up working directory and notebook path

In [ ]:
import os

# Change paths according to your needs.
if os.getenv("COLAB_RELEASE_TAG"):
  WORKING_DIR = "/content/drive/My Drive/deepxps"
  NOTEBOOK_PATH = "/content/drive/My Drive/deepxps/xpsdeeplearning/notebooks/predict.ipynb"

else:
  WORKING_DIR = r"C:/Users/pielsticker/Lukas/MPI-CEC/Projects/deepxps"
  NOTEBOOK_PATH = r"C:/Users/pielsticker/Lukas/MPI-CEC/Projects/deepxps/xpsdeeplearning/notebooks/predict.ipynb"

os.chdir(WORKING_DIR)

### Mount google drive

In [ ]:
# This cell only needs to be run when you are working on Google Colab.
# Mount drive
from google.colab import drive
drive.mount("/content/drive")

### Install packages and import modules

In [ ]:
%%capture
# Install packages if needed
if "google.colab" in str(get_ipython()):
     %pip install ./xpsdeeplearning

# Import standard modules
import datetime
import numpy as np
import pytz
import importlib
import matplotlib.pyplot as plt

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Import tensorflow
import tensorflow as tf
from tensorflow.keras import backend as K

### Set seeds and restart session to ensure reproducibility

In [ ]:
def reset_seeds_and_session(seed=1):
   os.environ["PYTHONHASHSEED"]=str(seed)
   tf.random.set_seed(seed)
   np.random.seed(seed)

   session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                           inter_op_parallelism_threads=1)
   sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                               config=session_conf)
   tf.compat.v1.keras.backend.set_session(sess)

reset_seeds_and_session(seed=1)

### Check TensorFlow version

In [ ]:
f"TF version: {tf.__version__}."

## Predict on new data set

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print("\n Modules were reloaded.")
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print("Modules were loaded.")

### Set up the parameters & folder structure

In [ ]:
time = datetime.datetime.now().astimezone(pytz.timezone("Europe/Berlin")).strftime("%Y%m%d_%Hh%Mm")
exp_name = "test"

clf = classifier.Classifier(time = time,
                            exp_name = exp_name,
                            task = "regression",
                            intensity_only = True)

### Load and inspect the data

In [ ]:
input_filepath = r"/content/drive/My Drive/deepxps/datasets/20220624_Ni_linear_combination_small_gas_phase.h5"
#input_filepath = r"C:/Users/pielsticker/Downloads/FeCo/20210903_CoFe_combined_without_auger_peaks.h5"

train_test_split = 0.99
train_val_split = 0
no_of_examples = 100#000 #180


X_train, X_val, X_test, y_train, y_val, y_test,\
    names_train, names_val, names_test =\
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split)

# Check how the examples are distributed across the classes.
class_distribution = clf.datahandler.check_class_distribution(clf.task)
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 10, dataset = "test")

### Optional: Continue with 10-point average of last values (cutoff: 5 eV on each side)

In [ ]:
for dataset in [clf.datahandler.X,
                clf.datahandler.X_train,
                clf.datahandler.X_val,
                clf.datahandler.X_test]:
    for arr in dataset:
        arr[:100,:] = np.average(arr[100:110,:],
                                 axis=0)
        arr[-100:,:] = np.average(arr[-110:-100,:],
                                  axis=0)

In [ ]:
clf.plot_random(no_of_spectra = 10, dataset = "test")

### Load and compile the model

In [ ]:
clf.load_model(model_path = "/content/drive/My Drive/deepxps/runs/20231117_10h57m_Ni_linear_combination_normalized_inputs_small_gas_phase_200k/model")

### Plot summary and save model plot.


In [ ]:
clf.summary()
clf.save_and_print_model_image()

### Evaluate on test data

In [ ]:
clf.logging.hyperparams["batch_size"] = 32

if clf.task == "classification":
    score = clf.evaluate()
    test_loss, test_accuracy = score[0], score[1]
    print("Test loss: " + str(np.round(test_loss, decimals=8)))
    print("Test accuracy: " + str(np.round(test_accuracy, decimals=3)))
elif clf.task == "regression":
    test_loss = clf.evaluate()
    print("Test loss: " + str(np.round(test_loss, decimals=8)))

###  Predict on train & test data

In [ ]:
pred_train, pred_test = clf.predict()
if clf.task == "classification":
    pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions on random test samples

In [ ]:
clf.plot_random(no_of_spectra = 15, dataset = "test", with_prediction = True)

In [ ]:
clf.datahandler.plot_spectra(no_of_spectra=20, dataset="test", indices=list(range(20)), with_prediction=True)


### Show the worst predictions on the test samples

In [ ]:
clf.show_worst_predictions(no_of_spectra = 10)

### Save data

In [ ]:
#clf.save_hyperparams()
clf.pickle_results()

## Check where and why the predictions fail

### Show worst predictions for single spectra

In [ ]:
clf.show_worst_predictions(no_of_spectra = 10, kind = "single")

#### Show worst predictions for different loss thresholds (single spectra)


In [ ]:
threshold = 0.2
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "single",
                           threshold = threshold)

In [ ]:
threshold = 0.1
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "single",
                           threshold = threshold)

In [ ]:
threshold = 0.05
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "single",
                           threshold = threshold)

In [ ]:
threshold = 0.02
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "single",
                           threshold = threshold)

In [ ]:
threshold = 0.01
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "single",
                           threshold = threshold)

### Show worst predictions for linearly combined spectra

In [ ]:
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb")

#### Show worst predictions for different loss thresholds (linearly combined spectra)

In [ ]:
threshold = 0.3
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.2
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.1
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.05
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.025
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.01
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

In [ ]:
threshold = 0.005
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "linear_comb",
                           threshold = threshold)

### Show worst predictions for all

In [ ]:
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all")

### Show worst predictions for different loss thresholds (all spectra)

In [ ]:
threshold = 0.3
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.2
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.1
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.05
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.025
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.01
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.005
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.001
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.0005
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.00025
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.0001
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

In [ ]:
threshold = 0.00001
clf.show_worst_predictions(no_of_spectra = 10,
                           kind = "all",
                           threshold = threshold)

## Remove empty model directory

In [ ]:
import shutil
shutil.rmtree(clf.logging.model_dir)

del(clf.logging.model_dir)

## Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=["application/javascript"])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is ".ipynb", output_file is ".html"
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, "w", encoding="utf-8").write(output)

import time
import os

time.sleep(20)
save_notebook()
print("Notebook saved!")
time.sleep(30)
output_file = os.path.join(clf.logging.log_dir,
                           "predict_out.html")
output_HTML(NOTEBOOK_PATH, output_file)
print("HTML file saved!")